In [1]:
from Functions import *

In [2]:
#7 Multilayer network ---------------------------------------------------------------------------
# As the data was complex, I have written the code to obtain a dataframe with the interactions
my_path="./data/Projects/INTERTIDAL/"

def read_adj_matrix(filename, interaction_type):
    """
    Read multiplex adjacency file and return edge list dataframe.
    interaction_type = 'T', 'NTp', or 'NTn'
    """
    df = pd.read_csv(filename, sep="\t", header=0)
    
    # First two cols: id, name
    ids = df.iloc[:,0]
    names = df.iloc[:,1]
    
    # Adjacency values start from col index 2
    adj = df.iloc[:,2:].values
    col_ids = df.columns[2:].astype(int)  # target species IDs
    
    edges = []
    for i, source_id in enumerate(ids):
        for j, target_id in enumerate(col_ids):
            if adj[i,j] == 1:
                edges.append([source_id, target_id, interaction_type])
    
    return pd.DataFrame(edges, columns=["source", "target", "interaction"])

ftype="TI" #TI,NTIpos, NTIneg
filename=my_path+"chilean_"+ftype+".txt"
edges_T   = read_adj_matrix(filename, ftype)
ftype="NTIpos" #TI,NTIpos, NTIneg
filename=my_path+"chilean_"+ftype+".txt"
edges_NTp = read_adj_matrix(filename, "NTp")
ftype="NTIneg" #TI,NTIpos, NTIneg
filename=my_path+"chilean_"+ftype+".txt"
edges_NTn = read_adj_matrix(filename, "NTn")

# Combine all edges: This is the interaction dataframe
edges_all = pd.concat([edges_T, edges_NTp, edges_NTn], ignore_index=True)

# Load metadata
meta = pd.read_csv(my_path+"chilean_metadata.csv")
# Merge source info
edges_all = edges_all.merge(
    meta[["Spec","Species names"]],
    left_on="source", right_on="Spec", how="left"
).rename(columns={"Species names":"source_name"}).drop(columns="Spec")

# Merge target info
edges_all = edges_all.merge(
    meta[["Spec","Species names"]],
    left_on="target", right_on="Spec", how="left"
).rename(columns={"Species names":"target_name"}).drop(columns="Spec")

#
MDG = nx.MultiDiGraph() #this is the graph with multiple itneractions

# Add nodes (with labels if you want)
for _, row in edges_all.iterrows():
    MDG.add_node(row["source"], label=row["source_name"])
    MDG.add_node(row["target"], label=row["target_name"])

# Add edges with interaction type
for _, row in edges_all.iterrows():
    MDG.add_edge(row["source"], row["target"], interaction=row["interaction"])

#if you want you can add the other rest of metadata of nodes as node atttributes
#this creates a dictinary where each node has all the attributes
node_attrs = meta.set_index("Spec")[["trophic", "Phyllum", "BodyMass"]].to_dict(orient="index")
#then add the attributes to each node: we insert the dictionary of attributes inside the network container
nx.set_node_attributes(MDG, node_attrs)    